# Load fact_orders từ Silver lên Gold

## Mục tiêu
- Load dữ liệu từ Silver.fact_orders → Gold.fact_orders
- Không cần cleaning, chỉ load trực tiếp
- Đảm bảo data quality cho Gold layer

## Quy trình 3 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Load to Gold** - Load vào Gold database
3. **Create Dictionary** - Tạo data dictionary


## Bước 1: Import và Setup


In [1]:
# Import các thư viện cần thiết
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-18 00:12:10


## Bước 2: Load dữ liệu từ Silver


In [2]:
# Load dữ liệu fact_orders từ Silver database
print("=== LOADING FACT_ORDERS FROM SILVER ===")

# Load toàn bộ bảng fact_orders
orders_df = pd.read_sql("SELECT * FROM fact_orders", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {orders_df.shape}")
print(f"Columns: {list(orders_df.columns)}")
print(f"Memory usage: {orders_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(orders_df.head(3))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(orders_df.dtypes)

# Kiểm tra null values
print("\n=== NULL VALUES CHECK ===")
null_summary = orders_df.isnull().sum()
print("Null values per column:")
for col, null_count in null_summary.items():
    if null_count > 0:
        print(f"  {col}: {null_count} ({null_count/len(orders_df)*100:.1f}%)")
    else:
        print(f"  {col}: {null_count} (0.0%)")


=== LOADING FACT_ORDERS FROM SILVER ===
Shape: (40236, 25)
Columns: ['order_id', 'system_id', 'shop_id', 'order_link', 'link_confirm_order', 'order_currency', 'total_price', 'total_price_after_sub_discount', 'total_discount', 'total_quantity', 'items_length', 'tax', 'shipping_fee', 'money_to_collect', 'is_free_shipping', 'customer_id', 'page_id', 'warehouse_id', 'shipping_id', 'payment_id', 'status', 'status_name', 'inserted_at', 'date_id', 'updated_at']
Memory usage: 40.63 MB

=== SAMPLE DATA ===
  order_id  system_id    shop_id  \
0    40616      40616  230361475   
1    40615      40615  230361475   
2    40614      40614  230361475   

                                          order_link  \
0  https://pos.pages.fm/shop/230361475/order?orde...   
1  https://pos.pages.fm/shop/230361475/order?orde...   
2  https://pos.pages.fm/shop/230361475/order?orde...   

                                  link_confirm_order order_currency  \
0  https://order.pke.gg/tracking?id=RyvKXPdNWkGlh...    

## Bước 3: Load vào Gold Database


In [3]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import BigInteger, Integer, String, Text, DateTime, Float

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {}

# ID fields - String (để xử lý mixed data types)
id_fields = ["order_id", "customer_id", "shop_id", "page_id", "warehouse_id", "shipping_id", "payment_id", "date_id"]
for field in id_fields:
    if field in orders_df.columns:
        dtype_mapping[field] = String(100)

# String fields - VARCHAR(255)
string_fields = ["order_status", "payment_status", "shipping_status", "order_note"]
for field in string_fields:
    if field in orders_df.columns:
        dtype_mapping[field] = String(255)

# Numeric fields
float_fields = ["total_amount", "discount_amount", "shipping_fee", "tax_amount"]
for field in float_fields:
    if field in orders_df.columns:
        dtype_mapping[field] = Float()

# Integer fields
int_fields = ["item_count"]
for field in int_fields:
    if field in orders_df.columns:
        dtype_mapping[field] = Integer()

# DateTime fields
datetime_fields = ["order_date", "created_at", "updated_at"]
for field in datetime_fields:
    if field in orders_df.columns:
        dtype_mapping[field] = DateTime()

print(f"Schema mapping defined for {len(dtype_mapping)} columns")

# Load vào Gold database
table_name = "fact_orders"

try:
    orders_df.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {orders_df.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT * FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:\n{sample_db}")
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

print(f"\nLoad fact_orders completed successfully!")
print(f"Ready for EDA and analytics")


=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 10 columns
Đã load 40236 records vào Gold: fact_orders
Verification: 40236 records trong database
Schema: 10 columns với explicit typing

Sample data từ Gold database:
  order_id  system_id    shop_id  \
0    40616      40616  230361475   
1    40615      40615  230361475   
2    40614      40614  230361475   
3    40613      40613  230361475   
4    40612      40612  230361475   

                                          order_link  \
0  https://pos.pages.fm/shop/230361475/order?orde...   
1  https://pos.pages.fm/shop/230361475/order?orde...   
2  https://pos.pages.fm/shop/230361475/order?orde...   
3  https://pos.pages.fm/shop/230361475/order?orde...   
4  https://pos.pages.fm/shop/230361475/order?orde...   

                                  link_confirm_order order_currency  \
0  https://order.pke.gg/tracking?id=RyvKXPdNWkGlh...            VND   
1  https://order.pke.gg/tracking?id=RyvKXPdNWkGlh...            VND   
2  ht

## Bước 4: Tạo Data Dictionary cho Gold


In [4]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Khóa
        "order_id": "Mã định danh đơn hàng duy nhất (khóa chính)",
        "customer_id": "Mã khách hàng (khóa ngoại đến dim_customers)",
        "shop_id": "Mã cửa hàng (khóa ngoại đến dim_shops)",
        "page_id": "Mã trang đặt hàng (khóa ngoại đến dim_order_pages)",
        "warehouse_id": "Mã kho hàng (khóa ngoại đến dim_order_warehouses)",
        "shipping_id": "Mã vận chuyển (khóa ngoại đến dim_order_shipping)",
        "payment_id": "Mã thanh toán (khóa ngoại đến dim_order_payments)",
        "date_id": "Mã ngày (khóa ngoại đến dim_date)",
        
        # Thông tin đơn hàng
        "order_status": "Trạng thái đơn hàng (pending, confirmed, shipped, delivered, cancelled)",
        "payment_status": "Trạng thái thanh toán (pending, paid, failed, refunded)",
        "shipping_status": "Trạng thái vận chuyển (pending, shipped, delivered, returned)",
        "order_note": "Ghi chú đơn hàng",
        
        # Số lượng và giá trị
        "item_count": "Số lượng sản phẩm trong đơn hàng",
        "total_amount": "Tổng giá trị đơn hàng (VND)",
        "discount_amount": "Số tiền giảm giá (VND)",
        "shipping_fee": "Phí vận chuyển (VND)",
        "tax_amount": "Thuế VAT (VND)",
        
        # Thời gian
        "order_date": "Ngày đặt hàng",
        "created_at": "Thời gian tạo bản ghi đơn hàng",
        "updated_at": "Thời gian cập nhật bản ghi cuối cùng"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in orders_df.columns:
    col_info = {
        "table_name": "fact_orders",
        "column_name": col,
        "dtype": str(orders_df[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": orders_df[col].isnull().sum(),
        "null_pct": round(orders_df[col].isnull().mean() * 100, 2),
        "unique_count": orders_df[col].nunique(),
        "sample_values": str(orders_df[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: fact_orders")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)


=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 25 columns

=== DATA DICTIONARY ===
     table_name                     column_name           dtype      sql_type  \
0   fact_orders                        order_id          object  VARCHAR(100)   
1   fact_orders                       system_id           int64   Not defined   
2   fact_orders                         shop_id           int64  VARCHAR(100)   
3   fact_orders                      order_link          object   Not defined   
4   fact_orders              link_confirm_order          object   Not defined   
5   fact_orders                  order_currency          object   Not defined   
6   fact_orders                     total_price         float64   Not defined   
7   fact_orders  total_price_after_sub_discount           int64   Not defined   
8   fact_orders                  total_discount           int64   Not defined   
9   fact_orders                  total_quantity           int64   Not defined   
10  f